In [ ]:
import pandas as pd

# Mount Google Drive to access files from it
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/NVIDIA_MASTER.csv')
display(df.head())

In [ ]:
import zipfile
import os

zip_file_path = '/content/drive/MyDrive/NVIDIA Project Files Team 4.zip'
extraction_path = '/content/drive/MyDrive/NVIDIA_unzipped_files'

# Create the directory for extraction if it doesn't exist
os.makedirs(extraction_path, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

print(f"'{zip_file_path}' unzipped to '{extraction_path}'")
print(f"Contents of '{extraction_path}':")
print(os.listdir(extraction_path))

In [ ]:
import os

# List the contents of your Google Drive's root folder
print(os.listdir('/content/drive/MyDrive'))

# If your file is in a subfolder, you might need to adjust the path:
# print(os.listdir('/content/drive/MyDrive/Your_Folder_Name'))

In [ ]:
df = df.dropna()
display(df.head())

In [ ]:
df = df.drop('Product Name', axis=1)
display(df.head())

In [ ]:
aggregated_df = df.groupby(['Region', 'Product Category', 'Date']).agg({
    'Sales Revenue (USD)': 'sum',
    'Units Sold': 'sum',
    'Customer Satisfaction': 'mean',
    'Marketing Spend (USD)': 'sum',
    'Discount Percentage (%)': 'mean',
    'Return Rate (%)': 'mean',
    'AI/ML Adoption Rate (%)': 'mean',
    'Ad Campaign Effectiveness': 'mean',
    'Customer Retention Rate (%)': 'mean',
    'Market Share (%)': 'mean',
    'Total_Revenue': 'sum'
}).reset_index()

display(aggregated_df.head())

In [ ]:
# Group by 'Region' and 'Product Category' and check for unique dates
is_unique_date = aggregated_df.groupby(['Region', 'Product Category'])['Date'].nunique() == aggregated_df.groupby(['Region', 'Product Category'])['Date'].size()

# Check if all combinations have unique dates
if is_unique_date.all():
    print("All Region and Product Category combinations have unique dates.")
else:
    print("Some Region and Product Category combinations have duplicate dates.")
    # Optionally, display the combinations with duplicate dates
    print("Combinations with duplicate dates:")
    print(is_unique_date[~is_unique_date])

In [ ]:
import os

# Create a directory to save the CSV files in Google Drive
output_dir = '/content/drive/MyDrive/NVIDIA/aggregated_dataframes'
os.makedirs(output_dir, exist_ok=True)

# Get unique combinations of Region and Product Category
unique_combinations = aggregated_df.groupby(['Region', 'Product Category']).groups.keys()

# Iterate through unique combinations and save each as a CSV
for region, product_category in unique_combinations:
    # Filter the aggregated_df for the current combination
    filtered_df = aggregated_df[(aggregated_df['Region'] == region) & (aggregated_df['Product Category'] == product_category)]

    # Create a filename based on the combination
    filename = f"{region}_{product_category}.csv".replace(" ", "_").replace("/", "_") # Replace spaces and slashes for valid filenames
    filepath = os.path.join(output_dir, filename)

    # Save the filtered dataframe to CSV
    filtered_df.to_csv(filepath, index=False)
    print(f"Saved {filename}")

print(f"\nAll aggregated dataframes saved to {output_dir}")

# Task
Analyze time series data from multiple CSV files located in "/content/drive/MyDrive/NVIDIA/aggregated_dataframes", checking for seasonality, plotting ACF and PACF, performing White Noise and stationarity tests, and summarizing the results in a table.

## List files

### Subtask:
Get the list of all CSV files in the specified directory.


**Reasoning**:
Get the list of all CSV files in the specified directory.



In [ ]:
import os

output_dir = '/content/drive/MyDrive/NVIDIA/aggregated_dataframes'
csv_files = [f for f in os.listdir(output_dir) if f.endswith('.csv')]
print(csv_files)

## Iterate and load

### Subtask:
Loop through each CSV file, load it into a pandas DataFrame, and convert the 'Date' column to datetime objects.


**Reasoning**:
Loop through each CSV file, load it into a pandas DataFrame, convert the 'Date' column to datetime objects, and store the preprocessed DataFrames in a dictionary.



In [ ]:
dataframes = {}

for filename in csv_files:
    filepath = os.path.join(output_dir, filename)
    df_temp = pd.read_csv(filepath)
    df_temp['Date'] = pd.to_datetime(df_temp['Date'])
    key_name = filename.replace('.csv', '')
    dataframes[key_name] = df_temp

# Display the keys of the dictionary and the head of one of the dataframes as a check
print("Loaded dataframes:")
print(dataframes.keys())
print("\nHead of one dataframe:")
display(list(dataframes.values())[0].head())

## Perform time series analysis

### Subtask:
For each DataFrame, perform the requested time series analysis:
- Check for seasonality.
- Plot ACF and PACF.
- Perform White Noise test.
- Perform Stationarity test (e.g., Augmented Dickey-Fuller test).


**Reasoning**:
Iterate through each dataframe in the `dataframes` dictionary and perform the requested time series analysis steps (seasonality check, ACF/PACF plots, White Noise test, and Stationarity test) on the 'Sales Revenue (USD)' column.



In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.diagnostic import acorr_ljungbox
import matplotlib.pyplot as plt

results = {}

for name, df in dataframes.items():
    print(f"Analyzing {name}:")
    ts = df['Sales Revenue (USD)']

    # 1. Visual inspection for seasonality (Plotting the time series)
    plt.figure(figsize=(12, 6))
    plt.plot(df['Date'], ts)
    plt.title(f'{name} - Sales Revenue (USD) Over Time')
    plt.xlabel('Date')
    plt.ylabel('Sales Revenue (USD)')
    plt.show()

    # 2. Plot ACF and PACF
    fig, axes = plt.subplots(1, 2, figsize=(16, 4))
    plot_acf(ts, ax=axes[0])
    plot_pacf(ts, ax=axes[1])
    axes[0].set_title(f'{name} - ACF')
    axes[1].set_title(f'{name} - PACF')
    plt.show()

    # 3. Perform White Noise test (Ljung-Box test)
    ljung_box_result = acorr_ljungbox(ts, lags=[10], return_df=True)
    ljung_box_pvalue = ljung_box_result['lb_pvalue'].iloc[0]
    print(f"Ljung-Box test p-value: {ljung_box_pvalue:.4f}")
    white_noise_status = "Not White Noise" if ljung_box_pvalue < 0.05 else "White Noise"
    print(f"Conclusion: The time series is likely {white_noise_status}.\n")

    # 4. Perform Stationarity test (Augmented Dickey-Fuller test)
    adf_result = adfuller(ts)
    adf_statistic = adf_result[0]
    adf_pvalue = adf_result[1]
    print(f"ADF Statistic: {adf_statistic:.4f}")
    print(f"ADF p-value: {adf_pvalue:.4f}")
    stationarity_status = "Stationary" if adf_pvalue < 0.05 else "Non-Stationary"
    print(f"Conclusion: The time series is likely {stationarity_status}.\n")

    results[name] = {
        'Ljung-Box p-value': ljung_box_pvalue,
        'White Noise Conclusion': white_noise_status,
        'ADF Statistic': adf_statistic,
        'ADF p-value': adf_pvalue,
        'Stationarity Conclusion': stationarity_status
    }

# Summarize results in a table
results_df = pd.DataFrame.from_dict(results, orient='index')
print("Summary of Time Series Analysis Results:")
display(results_df)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
import matplotlib.pyplot as plt

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')

results = {}

for name, df in dataframes.items():
    print(f"Analyzing time series for {name}...")
    df_temp = df.copy()
    df_temp['Date'] = pd.to_datetime(df_temp['Date'])
    df_temp = df_temp.set_index('Date').sort_index()

    # Select the target variable and resample to a consistent frequency (e.g., daily 'D')
    # Aggregate by summing sales revenue for each day if multiple entries exist for the same date
    ts = df_temp['Sales Revenue (USD)'].resample('D').sum().fillna(method='ffill') # Forward fill missing daily data

    # Decompose time series into trend, seasonal, and residual components
    # Adjust model based on assumption (additive or multiplicative) - additive is often a good start
    try:
        decomposition = seasonal_decompose(ts, model='additive')

        # Plot the decomposed time series components
        fig, axes = plt.subplots(4, 1, figsize=(12, 10), sharex=True)
        axes[0].plot(ts, label='Original')
        axes[0].set_title(f'{name} - Original Time Series')
        axes[0].legend()

        axes[1].plot(decomposition.trend, label='Trend')
        axes[1].set_title(f'{name} - Trend Component')
        axes[1].legend()

        axes[2].plot(decomposition.seasonal, label='Seasonal')
        axes[2].set_title(f'{name} - Seasonal Component')
        axes[2].legend()

        axes[3].plot(decomposition.resid, label='Residuals')
        axes[3].set_title(f'{name} - Residual Component')
        axes[3].legend()

        plt.xlabel('Date')
        plt.tight_layout()
        plt.show()

        # Analyze residuals: ACF and PACF plots
        print(f"Analyzing residuals for {name}...")
        residuals = decomposition.resid.dropna() # Drop NaNs for residual analysis

        if not residuals.empty:
            fig, axes = plt.subplots(1, 2, figsize=(12, 4))
            plot_acf(residuals, ax=axes[0], lags=min(40, len(residuals)//2 - 1)) # Plot ACF of residuals
            axes[0].set_title(f'{name} - Residuals ACF')
            plot_pacf(residuals, ax=axes[1], lags=min(40, len(residuals)//2 - 1)) # Plot PACF of residuals
            axes[1].set_title(f'{name} - Residuals PACF')
            plt.tight_layout()
            plt.show()

            # Ljung-Box test for white noise in residuals
            # Perform test on a range of lags
            ljung_box_result = acorr_ljungbox(residuals, lags=[10], return_df=True)
            ljung_box_pvalue = ljung_box_result.iloc[0, 1]

            white_noise_conclusion = "White Noise" if ljung_box_pvalue > 0.05 else "Not White Noise"

            print(f"Ljung-Box test p-value for {name}: {ljung_box_pvalue:.4f}")
            print(f"Conclusion: Residuals are likely {white_noise_conclusion}")

            results[name] = {
                'Ljung-Box p-value': ljung_box_pvalue,
                'White Noise Conclusion': white_noise_conclusion
            }
        else:
            print(f"Residuals are empty for {name}, skipping residual analysis.")
            results[name] = {
                'Ljung-Box p-value': np.nan,
                'White Noise Conclusion': 'N/A (Empty Residuals)'
            }


    except Exception as e:
        print(f"Could not perform time series decomposition for {name}: {e}")
        results[name] = {
            'Ljung-Box p-value': np.nan,
            'White Noise Conclusion': f'Error during decomposition: {e}'
        }


print("\nTime series analysis complete for all dataframes.")

# Robustness and Stability Check

## Present summary

### Subtask:
Display the summarized results for each file.


**Reasoning**:
Display the `results_df` DataFrame which contains the summary of the time series analysis for each file.



In [ ]:
display(results_df)

## Summary:

### Data Analysis Key Findings

*   Most of the time series analyzed were found to be White Noise according to the Ljung-Box test (p-value > 0.05), except for "Middle_East_OEM" and "North_America_Data_Center" which were identified as "Not White Noise".
*   All time series were concluded to be Stationary based on the Augmented Dickey-Fuller (ADF) test results (p-value < 0.05 for all).
*   The analysis involved loading 24 CSV files, converting their 'Date' column to datetime objects, plotting time series, ACF, and PACF for each, and performing Ljung-Box and ADF tests.

### Insights or Next Steps

*   Given that most series are White Noise and all are stationary, simple forecasting models might be sufficient, but the "Not White Noise" series ("Middle_East_OEM" and "North_America_Data_Center") might benefit from specific time series models like ARIMA or SARIMA, provided the visual inspection of plots suggests specific patterns not captured by the tests alone.
*   Investigate the underlying reasons why "Middle_East_OEM" and "North_America_Data_Center" series are not white noise, which could indicate specific patterns or external factors influencing their sales revenue.


# Task
Load the aggregated dataframes from the CSV files located in "/content/drive/MyDrive/NVIDIA/aggregated_dataframes", train and test an XGBoost model using 'Sales Revenue' as the dependent variable to forecast sales revenue for the next 1 year for each dataframe, calculate confidence intervals for the forecasts, and visualize the historical data, forecasted sales revenue, and confidence intervals for each dataframe in a professional presentation-ready format with legends.

## Load data

### Subtask:
Load the aggregated dataframes from the previously saved CSV files.


**Reasoning**:
Loop through each CSV file, load it into a pandas DataFrame, convert the 'Date' column to datetime objects, and store the preprocessed DataFrames in a dictionary.



In [ ]:
dataframes = {}

for filename in csv_files:
    filepath = os.path.join(output_dir, filename)
    df_temp = pd.read_csv(filepath)
    df_temp['Date'] = pd.to_datetime(df_temp['Date'])
    key_name = filename.replace('.csv', '')
    dataframes[key_name] = df_temp

# Display the keys of the dictionary and the head of one of the dataframes as a check
print("Loaded dataframes:")
print(dataframes.keys())
display(list(dataframes.values())[0].head())

## Prepare data for forecasting

### Subtask:
For each dataframe, prepare the data for time series forecasting by creating features and splitting the data into training and testing sets.


**Reasoning**:
Iterate through each dataframe, set the 'Date' column as the index, define features and target, and split the data into training and testing sets based on the date threshold.



In [ ]:
forecast_data = {}
date_threshold = '2019-09-23'

for name, df in dataframes.items():
    df = df.set_index('Date')

    target = df['Sales Revenue (USD)']
    features = df[['Marketing Spend (USD)', 'Ad Campaign Effectiveness']]

    X_train = features[features.index <= date_threshold]
    X_test = features[features.index > date_threshold]
    y_train = target[target.index <= date_threshold]
    y_test = target[target.index > date_threshold]

    forecast_data[name] = {
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test,
        'date_threshold': date_threshold
    }

# Display the keys of the new dictionary and the head of one of the training sets as a check
print("Prepared data for forecasting:")
print(forecast_data.keys())
print("\nHead of one training set (features):")
display(list(forecast_data.values())[0]['X_train'].head())
print("\nHead of one training set (target):")
display(list(forecast_data.values())[0]['y_train'].head())
print("\nHead of one testing set (features):")
display(list(forecast_data.values())[0]['X_test'].head())
print("\nHead of one testing set (target):")
display(list(forecast_data.values())[0]['y_test'].head())

## Train xgboost model

### Subtask:
Train an XGBoost model on the training data for each dataframe.


**Reasoning**:
Iterate through each dataframe in the `forecast_data` dictionary, train an XGBoost model on the training data, and store the trained model.



In [ ]:
from xgboost import XGBRegressor

models = {}

for name, data in forecast_data.items():
    print(f"Training XGBoost model for {name}...")
    X_train = data['X_train']
    y_train = data['y_train']

    model = XGBRegressor(random_state=42)
    model.fit(X_train, y_train)

    models[name] = model
    print(f"Finished training for {name}")

print("\nAll XGBoost models trained.")

## Forecast future sales

### Subtask:
Use the trained XGBoost model to forecast sales revenue for the next 1 year for each dataframe.


**Reasoning**:
Iterate through each trained model and its corresponding test features to generate sales revenue forecasts.



In [ ]:
forecasts = {}

for name, model in models.items():
    print(f"Generating forecasts for {name}...")
    X_test = forecast_data[name]['X_test']
    forecast = model.predict(X_test)
    forecasts[name] = pd.Series(forecast, index=X_test.index)
    print(f"Finished forecasting for {name}")

print("\nAll forecasts generated.")

## Calculate confidence intervals

### Subtask:
Calculate confidence intervals for the forecasted sales revenue.


**Reasoning**:
Iterate through the trained models and forecast data, calculate residuals, their standard deviation, and then compute the confidence intervals for the forecasts based on a 95% confidence level. Store these confidence intervals.



In [ ]:
import numpy as np
from scipy.stats import norm

confidence_intervals = {}
confidence_level = 0.95
z_score = norm.ppf((1 + confidence_level) / 2) # Z-score for 95% confidence

for name, model in models.items():
    print(f"Calculating confidence intervals for {name}...")
    X_train = forecast_data[name]['X_train']
    y_train = forecast_data[name]['y_train']
    X_test = forecast_data[name]['X_test']

    # Calculate residuals on the training data
    residuals = y_train - model.predict(X_train)

    # Calculate the standard deviation of the residuals
    residual_std = residuals.std()

    # Calculate the margin of error
    margin_of_error = z_score * residual_std

    # Calculate the lower and upper bounds of the confidence interval for the forecasts
    lower_bound = forecasts[name] - margin_of_error
    upper_bound = forecasts[name] + margin_of_error

    confidence_intervals[name] = pd.DataFrame({
        'lower_bound': lower_bound,
        'upper_bound': upper_bound
    }, index=forecasts[name].index)

    print(f"Finished calculating confidence intervals for {name}")

print("\nConfidence intervals for all forecasts calculated.")

## Visualize forecasts

### Subtask:
For each dataframe, visualize the historical data, the forecasted sales revenue, and the confidence interval, ensuring the plots are suitable for a presentation with legends.


**Reasoning**:
Iterate through each dataframe and plot the historical data, forecasted sales, and confidence intervals.



In [ ]:
import matplotlib.pyplot as plt

for name in dataframes.keys():
    plt.figure(figsize=(12, 6))
    plt.plot(forecast_data[name]['y_train'].index, forecast_data[name]['y_train'], label='Historical Data')
    plt.plot(forecasts[name].index, forecasts[name], label='Forecast')
    plt.plot(confidence_intervals[name].index, confidence_intervals[name]['upper_bound'], label='Upper Bound', linestyle='--')
    plt.plot(confidence_intervals[name].index, confidence_intervals[name]['lower_bound'], label='Lower Bound', linestyle='--')
    plt.title(f'Sales Revenue Forecast for {name}')
    plt.xlabel('Date')
    plt.ylabel('Sales Revenue (USD)')
    plt.legend()
    plt.show()

## Summarize results

### Subtask:
Summarize the forecasting results, including model performance metrics and key observations for each dataframe.


**Reasoning**:
Calculate and display the MAE and RMSE for each dataframe's forecasts and summarize the results in a DataFrame.



In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

results_summary = {}

def mean_absolute_percentage_error(y_true, y_pred):
    """Calculates Mean Absolute Percentage Error (MAPE)."""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

for name in forecasts.keys():
    y_true = forecast_data[name]['y_test']
    y_pred = forecasts[name]

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = mean_absolute_percentage_error(y_true, y_pred)


    results_summary[name] = {
        'MAE': mae,
        'RMSE': rmse,
        'MAPE': mape
    }

summary_df = pd.DataFrame.from_dict(results_summary, orient='index')

print("Summary of Forecasting Performance (MAE, RMSE, and MAPE):")
display(summary_df)

## Summary:

### Data Analysis Key Findings

*   The data was successfully loaded from 24 CSV files, each representing aggregated data for a specific region and product category.
*   For each dataset, features ('Marketing Spend (USD)' and 'Ad Campaign Effectiveness') and the target variable ('Sales Revenue (USD)') were identified.
*   The data was split into training (up to '2019-09-23') and testing sets (after '2019-09-23') for each dataset.
*   An XGBoost Regressor model was trained on the training data for each of the 24 datasets.
*   Sales revenue forecasts for the next year were generated using the trained models on the testing data.
*   95% confidence intervals for the forecasts were calculated based on the residuals of the models on the training data.
*   Visualizations were created for each dataset, displaying historical data, forecasted sales revenue, and the calculated confidence intervals.
*   Model performance was evaluated using Mean Absolute Error (MAE) and Root Mean Squared Error (RMSE) on the test sets for each dataset.

### Insights or Next Steps

*   Analyze the MAE and RMSE values across different regions and product categories to identify which combinations have the most accurate forecasts and which require further model tuning or feature engineering.
*   Investigate the periods where the actual sales fall outside the 95% confidence intervals in the visualizations to understand potential drivers of unexpected sales fluctuations not captured by the current features.


In [ ]:
from xgboost import plot_tree
import matplotlib.pyplot as plt

# Visualize the first tree for each model
for name, model in models.items():
    print(f"Visualizing the first tree for {name}...")
    fig, ax = plt.subplots(figsize=(200, 100))
    plot_tree(model, tree_idx=0, ax=ax) # Changed num_trees to tree_idx
    ax.set_title(f"First Tree of XGBoost Model for {name}")
    plt.show()

## Present summary of forecasting results

### Subtask:
Display the summarized forecasting performance metrics for each dataframe.

**Reasoning**:
Display the `summary_df` DataFrame which contains the MAE, RMSE, and MAPE for each dataframe's sales revenue forecast. This table provides a clear overview of the model's performance across different regions and product categories.

In [ ]:
display(summary_df)

## Summary:

### Data Analysis Key Findings

* The data was successfully loaded from 24 CSV files, each representing aggregated data for a specific region and product category.
* For each dataset, features ('Marketing Spend (USD)' and 'Ad Campaign Effectiveness') and the target variable ('Sales Revenue (USD)') were identified.
* The data was split into training (up to '2019-09-23') and testing sets (after '2019-09-23') for each dataset.
* An XGBoost Regressor model was trained on the training data for each of the 24 datasets.
* Sales revenue forecasts for the next year were generated using the trained models on the testing data.
* 95% confidence intervals for the forecasts were calculated based on the residuals of the models on the training data.
* Visualizations were created for each dataset, displaying historical data, forecasted sales revenue, and the calculated confidence intervals, including the visualized decision trees for each model.
* Model performance was evaluated using Mean Absolute Error (MAE), Root Mean Squared Error (RMSE), and Mean Absolute Percentage Error (MAPE) on the test sets for each dataset.

### Insights or Next Steps

* **Compare performance across datasets:** Analyze the MAE, RMSE, and MAPE values in the summary table (`summary_df`) to identify which Region and Product Category combinations have the most accurate forecasts (lower MAE, RMSE, and MAPE) and which have higher errors, indicating potential areas for improvement.
* **Investigate high error datasets:** For datasets with high MAE, RMSE, and/or MAPE, explore the time series plots and the underlying data to understand if there are specific patterns, outliers, or external factors that the current features are not capturing.
* **Feature Engineering:** Consider creating additional relevant features that might improve the model's ability to capture patterns in the data, such as lag features, rolling statistics, or external economic indicators.
* **Model Tuning:** Experiment with different hyperparameters for the XGBoost model to potentially improve performance.
* **Explore other models:** If XGBoost performance is not satisfactory, consider exploring other time series forecasting models like ARIMA, SARIMA, or other machine learning models suitable for sequential data.
* **Refine Confidence Intervals:** The current confidence intervals are based on the standard deviation of the residuals from the training data. For a more robust analysis, consider exploring methods for calculating confidence intervals specifically designed for time series forecasts or tree-based models.
* **Domain Knowledge:** Incorporate domain knowledge about the specific regions and product categories to interpret the forecasting results and identify potential drivers of sales revenue that were not included in the model.

**Finish task**: Review the analysis and findings to draw conclusions about the sales revenue forecasting for the different regions and product categories.

## Summary:

### Data Analysis Key Findings

* The data was successfully loaded from 24 CSV files, each representing aggregated data for a specific region and product category.
* For each dataset, features ('Marketing Spend (USD)' and 'Ad Campaign Effectiveness') and the target variable ('Sales Revenue (USD)') were identified.
* The data was split into training (up to '2019-09-23') and testing sets (after '2019-09-23') for each dataset.
* An XGBoost Regressor model was trained on the training data for each of the 24 datasets.
* Sales revenue forecasts for the next year were generated using the trained models on the testing data.
* 95% confidence intervals for the forecasts were calculated based on the residuals of the models on the training data.
* Visualizations were created for each dataset, displaying historical data, forecasted sales revenue, and the calculated confidence intervals.
* Model performance was evaluated using Mean Absolute Error (MAE), Root Mean Squared Error (RMSE), and Mean Absolute Percentage Error (MAPE) on the test sets for each dataset.

### Insights or Next Steps

* **Compare performance across datasets:** Analyze the MAE, RMSE, and MAPE values in the summary table (`summary_df`) to identify which Region and Product Category combinations have the most accurate forecasts (lower MAE, RMSE, and MAPE) and which have higher errors, indicating potential areas for improvement.
* **Investigate high error datasets:** For datasets with high MAE, RMSE, and/or MAPE, explore the time series plots and the underlying data to understand if there are specific patterns, outliers, or external factors that the current features are not capturing.
* **Feature Engineering:** Consider creating additional relevant features that might improve the model's ability to capture patterns in the data, such as lag features, rolling statistics, or external economic indicators.
* **Model Tuning:** Experiment with different hyperparameters for the XGBoost model to potentially improve performance.
* **Explore other models:** If XGBoost performance is not satisfactory, consider exploring other time series forecasting models like ARIMA, SARIMA, or other machine learning models suitable for sequential data.
* **Refine Confidence Intervals:** The current confidence intervals are based on the standard deviation of the residuals from the training data. For a more robust analysis, consider exploring methods for calculating confidence intervals specifically designed for time series forecasts or tree-based models.
* **Domain Knowledge:** Incorporate domain knowledge about the specific regions and product categories to interpret the forecasting results and identify potential drivers of sales revenue that were not included in the model.

# ARIMA

12 month Holdout


In [ ]:
# Prepare data for ARIMA
arima_data = {}
date_threshold_arima = '2019-09-23'

for name, df in dataframes.items():
    # Set 'Date' as index
    df_arima = df.set_index('Date').sort_index()

    # Select only the target variable for ARIMA
    ts_arima = df_arima['Sales Revenue (USD)']

    # Split data into training and testing sets
    train_arima = ts_arima[ts_arima.index <= date_threshold_arima]
    test_arima = ts_arima[ts_arima.index > date_threshold_arima]

    arima_data[name] = {
        'train': train_arima,
        'test': test_arima
    }

# Display the keys of the new dictionary and the head of one of the training sets as a check
print("Prepared data for ARIMA:")
print(arima_data.keys())
print("\nHead of one training set:")
display(list(arima_data.values())[0]['train'].head())
print("\nHead of one testing set:")
display(list(arima_data.values())[0]['test'].head())

## Train ARIMA models

### Subtask:
Train an ARIMA model on the training data for each dataframe.

**Reasoning**:
Iterate through the prepared ARIMA data for each dataframe and train an ARIMA model on the training data. Since the optimal ARIMA parameters (p, d, q) can vary for each time series, a simple approach is to start with a common order (e.g., (5,1,0)) or implement a more sophisticated parameter selection process (e.g., using auto_arima or a grid search). For this task, we'll start with a basic order and note that this could be further optimized.

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

arima_models = {}
# Define the specified ARIMA order (p, d, q)
arima_order = (2, 1, 0)

for name, data in arima_data.items():
    print(f"Training ARIMA model for {name} with order {arima_order}...")
    train_data = data['train']

    try:
        # Fit ARIMA model using statsmodels
        model = ARIMA(train_data, order=arima_order)
        model_fit = model.fit()
        arima_models[name] = model_fit
        print(f"Finished training for {name}")
    except Exception as e:
        print(f"Could not train ARIMA model for {name}: {e}")
        arima_models[name] = None # Store None if training fails

print("\nAll ARIMA models trained (or attempted to train).")

## Forecast future sales with ARIMA

### Subtask:
Use the trained ARIMA models to forecast sales revenue for the next 1 year for each dataframe.

**Reasoning**:
Iterate through the trained ARIMA models and the test data for each dataframe to generate sales revenue forecasts.

In [ ]:
arima_forecasts = {}

for name, model in arima_models.items():
    if model: # Check if model training was successful
        print(f"Generating ARIMA forecasts for {name}...")
        test_data = arima_data[name]['test']
        # Forecast for the length of the test set
        forecast = model.forecast(steps=len(test_data))
        arima_forecasts[name] = forecast
        print(f"Finished forecasting for {name}")
    else:
        print(f"Skipping forecasting for {name} due to failed model training.")
        arima_forecasts[name] = None

print("\nAll ARIMA forecasts generated (or skipped).")

## Summarize ARIMA forecasting results

### Subtask:
Calculate and summarize the performance metrics (MAE, RMSE, MAPE) for the ARIMA forecasts and discuss the key findings and potential next steps.

**Reasoning**:
Calculate MAE, RMSE, and MAPE for the ARIMA forecasts for each dataframe and display the results in a table. Discuss the performance of the ARIMA models and suggest next steps, including comparing the ARIMA results to the XGBoost results.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

arima_results_summary = {}

def mean_absolute_percentage_error(y_true, y_pred):
    """Calculates Mean Absolute Percentage Error (MAPE)."""
    # Avoid division by zero for actual values that are zero
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # Replace 0s in y_true with a small epsilon to avoid division by zero
    y_true = np.where(y_true == 0, np.finfo(float).eps, y_true)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

for name in arima_forecasts.keys():
    if arima_forecasts[name] is not None:
        y_true = arima_data[name]['test']
        y_pred_series = arima_forecasts[name]

        # Explicitly set the index of the forecast to match the test data index
        y_pred_series.index = y_true.index

        # Align the series based on their index (which should now match)
        y_true_aligned, y_pred_aligned = y_true.align(y_pred_series, join='inner')

        # Ensure there are samples after alignment
        if len(y_true_aligned) > 0:
            mae = mean_absolute_error(y_true_aligned, y_pred_aligned)
            rmse = np.sqrt(mean_squared_error(y_true_aligned, y_pred_aligned))
            mape = mean_absolute_percentage_error(y_true_aligned, y_pred_aligned)

            arima_results_summary[name] = {
                'ARIMA MAE': mae,
                'ARIMA RMSE': rmse,
                'ARIMA MAPE': mape
            }
        else:
            print(f"No overlapping dates for {name} after alignment. Skipping metrics calculation.")
            arima_results_summary[name] = {
                'ARIMA MAE': np.nan,
                'ARIMA RMSE': np.nan,
                'ARIMA MAPE': np.nan
            }
    else:
         arima_results_summary[name] = {
            'ARIMA MAE': np.nan,
            'ARIMA RMSE': np.nan,
            'ARIMA MAPE': np.nan
        }


arima_summary_df = pd.DataFrame.from_dict(arima_results_summary, orient='index')

print("Summary of ARIMA Forecasting Performance (MAE, RMSE, and MAPE):")
display(arima_summary_df)

## Comparison and Summary

### Data Analysis Key Findings

* ARIMA models with a fixed order (2, 1, 0) were trained on the time series data for each of the 24 Region and Product Category combinations.
* Sales revenue forecasts for the test period (next year based on the split) were generated using the trained ARIMA models.
* 95% confidence intervals for the ARIMA forecasts were calculated.
* Visualizations were created for each dataset, displaying the historical data, actual future data, ARIMA forecasted sales revenue, and the calculated confidence intervals.
* Model performance was evaluated using Mean Absolute Error (MAE), Root Mean Squared Error (RMSE), and Mean Absolute Percentage Error (MAPE) on the test sets for each dataset.

### Insights or Next Steps

* **Compare ARIMA and XGBoost Performance:** Now that both XGBoost and ARIMA models have been trained and evaluated, compare the `summary_df` (XGBoost results) with the `arima_summary_df` (ARIMA results) to determine which model performed better for each specific time series. This comparison should be based on MAE, RMSE, and MAPE.
* **Analyze Performance Differences:** Investigate why one model might outperform the other for certain datasets. ARIMA models are often good at capturing temporal dependencies, while XGBoost, being a tree-based model, might be better at capturing non-linear relationships with the features (Marketing Spend and Ad Campaign Effectiveness).
* **Model Selection:** Based on the performance comparison, select the best model for forecasting sales revenue for each Region and Product Category combination.
* **Ensemble Methods:** Consider combining the forecasts from both the ARIMA and XGBoost models using ensemble techniques to potentially achieve even better forecasting accuracy.
* **Further ARIMA Tuning:** If ARIMA performance is not satisfactory, consider performing a more thorough analysis to determine the optimal (p, d, q) order for each time series. This could involve analyzing ACF/PACF plots, using information criteria (AIC, BIC), or implementing a grid search.
* **Review Confidence Intervals:** Examine the confidence intervals from both models in the visualizations. Wider intervals indicate greater uncertainty in the forecasts. Analyze if the actual future data falls within the confidence intervals for both models.
* **External Factors:** Revisit the possibility of incorporating external factors (beyond Marketing Spend and Ad Campaign Effectiveness) that might influence sales revenue, such as economic indicators, competitor actions, or seasonal events, which could improve the performance of both models.

**Finish task**: Based on the comparison and further analysis, finalize the sales revenue forecasts and present the findings.

## Calculate confidence intervals for ARIMA forecasts

### Subtask:
Calculate confidence intervals for the ARIMA forecasted sales revenue.

**Reasoning**:
Iterate through the trained ARIMA models and their corresponding forecasts to calculate the confidence intervals.

In [ ]:
arima_confidence_intervals = {}
confidence_level = 0.95 # 95% confidence interval

for name, model in arima_models.items():
    if model: # Check if model training was successful
        print(f"Calculating confidence intervals for {name}...")
        test_data = arima_data[name]['test']
        # Get forecast and confidence intervals
        forecast_results = model.get_forecast(steps=len(test_data))
        confidence_int = forecast_results.conf_int(alpha=1 - confidence_level)

        arima_confidence_intervals[name] = confidence_int
        print(f"Finished calculating confidence intervals for {name}")
    else:
        print(f"Skipping confidence interval calculation for {name} due to failed model training.")
        arima_confidence_intervals[name] = None

print("\nConfidence intervals for all ARIMA forecasts calculated (or skipped).")

## Visualize ARIMA forecasts

### Subtask:
For each dataframe, visualize the historical data, the ARIMA forecasted sales revenue, and the confidence interval.

**Reasoning**:
Iterate through each dataframe and plot the historical data, ARIMA forecasts, and confidence intervals.

In [ ]:
import matplotlib.pyplot as plt

for name in arima_data.keys():
    if arima_models[name] and arima_forecasts[name] is not None and arima_confidence_intervals[name] is not None:
        plt.figure(figsize=(12, 6))

        # Plot historical data
        plt.plot(arima_data[name]['train'].index, arima_data[name]['train'], label='Historical Data')
        plt.plot(arima_data[name]['test'].index, arima_data[name]['test'], label='Actual Future Data', color='orange')


        # Plot forecast and confidence intervals
        # Ensure the forecast and confidence interval indices match the test data index
        forecast_index = arima_data[name]['test'].index
        plt.plot(forecast_index, arima_forecasts[name], label='ARIMA Forecast', color='green')
        plt.fill_between(forecast_index,
                         arima_confidence_intervals[name].iloc[:, 0],
                         arima_confidence_intervals[name].iloc[:, 1], color='green', alpha=.2, label='95% Confidence Interval')

        plt.title(f'ARIMA Sales Revenue Forecast for {name}')
        plt.xlabel('Date')
        plt.ylabel('Sales Revenue (USD)')
        plt.legend()
        plt.show()
    else:
        print(f"Skipping visualization for {name} due to failed model training or forecasting.")

print("\nAll ARIMA forecasts visualized (or skipped).")

# ARIMAX

Using Marketing Spend as Independent Variable

In [ ]:
# Prepare data for ARIMAX
arimax_data = {}
date_threshold_arimax = '2019-09-23'

for name, df in dataframes.items():
    # Set 'Date' as index
    df_arimax = df.set_index('Date').sort_index()

    # Select the target variable and exogenous variable for ARIMAX
    ts_arimax = df_arimax['Sales Revenue (USD)']
    exog_arimax = df_arimax[['Marketing Spend (USD)']] # Using Marketing Spend as the exogenous variable

    # Split data into training and testing sets
    train_ts_arimax = ts_arimax[ts_arimax.index <= date_threshold_arimax]
    test_ts_arimax = ts_arimax[ts_arimax.index > date_threshold_arimax]
    train_exog_arimax = exog_arimax[exog_arimax.index <= date_threshold_arimax]
    test_exog_arimax = exog_arimax[exog_arimax.index > date_threshold_arimax]


    arimax_data[name] = {
        'train_ts': train_ts_arimax,
        'test_ts': test_ts_arimax,
        'train_exog': train_exog_arimax,
        'test_exog': test_exog_arimax
    }

# Display the keys of the new dictionary and the head of one of the training sets as a check
print("Prepared data for ARIMAX:")
print(arimax_data.keys())
print("\nHead of one training set (target):")
display(list(arimax_data.values())[0]['train_ts'].head())
print("\nHead of one training set (exogenous):")
display(list(arimax_data.values())[0]['train_exog'].head())
print("\nHead of one testing set (target):")
display(list(arimax_data.values())[0]['test_ts'].head())
print("\nHead of one testing set (exogenous):")
display(list(arimax_data.values())[0]['test_exog'].head())

## Comparison and Summary

### Data Analysis Key Findings

* ARIMAX models with a fixed ARIMA order (2, 1, 0) and 'Marketing Spend (USD)' as an exogenous variable were trained on the time series data for each of the 24 Region and Product Category combinations.
* Sales revenue forecasts for the test period (next year based on the split) were generated using the trained ARIMAX models and the corresponding exogenous data.
* 95% confidence intervals for the ARIMAX forecasts were calculated.
* Visualizations were created for each dataset, displaying the historical data, actual future data, ARIMAX forecasted sales revenue, and the calculated confidence intervals.
* Model performance was evaluated using Mean Absolute Error (MAE), Root Mean Squared Error (RMSE), and Mean Absolute Percentage Error (MAPE) on the test sets for each dataset.

### Insights or Next Steps

* **Compare Model Performance:** Now that forecasts from XGBoost, ARIMA, and ARIMAX models are available, compare the performance metrics (`summary_df`, `arima_summary_df`, and `arimax_summary_df`) to determine which model provides the most accurate forecasts for each specific time series. Consider MAE, RMSE, and MAPE for this comparison.
* **Analyze Exogenous Variable Impact:** Evaluate if the inclusion of 'Marketing Spend (USD)' as an exogenous variable in the ARIMAX model improved forecasting accuracy compared to the standard ARIMA model. This can be done by comparing the performance metrics of the ARIMA and ARIMAX models.
* **Model Selection:** Based on the comprehensive performance comparison across all three models, select the best model for forecasting sales revenue for each Region and Product Category combination.
* **Further Model Improvement:** For the selected best model(s), consider further tuning hyperparameters, exploring additional relevant exogenous variables, or investigating more advanced time series modeling techniques if needed to improve forecasting accuracy.
* **Review Confidence Intervals:** Examine the confidence intervals from all models in the visualizations. Wider intervals indicate greater uncertainty. Assess which model provides narrower confidence intervals while still capturing the actual future data points.
* **Business Context:** Interpret the forecasting results within the business context of each region and product category. Consider if the forecasts align with business expectations and if there are any insights from the models that can inform business strategies related to marketing spend or other factors.

**Finish task**: Based on the comparison and further analysis, finalize the sales revenue forecasts and present the findings, including a recommendation for the best forecasting approach for each time series.

## Train ARIMAX models

### Subtask:
Train an ARIMAX model on the training data for each dataframe, using 'Marketing Spend (USD)' as the exogenous variable.

**Reasoning**:
Iterate through the prepared ARIMAX data for each dataframe and train an ARIMAX model on the training time series data with the corresponding training exogenous variable. Similar to ARIMA, the optimal ARIMAX order and exogenous variables can be complex to determine automatically without libraries like `pmdarima`. For this task, we'll use a basic ARIMA order (2, 1, 0) and the specified exogenous variable.

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

arimax_models = {}
# Define a basic ARIMAX order (p, d, q) - this can be optimized
arimax_order = (2, 1, 0)

for name, data in arimax_data.items():
    print(f"Training ARIMAX model for {name} with order {arimax_order} and exogenous variable 'Marketing Spend (USD)'...")
    train_ts = data['train_ts']
    train_exog = data['train_exog']

    try:
        # Fit ARIMAX model using statsmodels
        model = ARIMA(train_ts, exog=train_exog, order=arimax_order)
        model_fit = model.fit()
        arimax_models[name] = model_fit
        print(f"Finished training for {name}")
    except Exception as e:
        print(f"Could not train ARIMAX model for {name}: {e}")
        arimax_models[name] = None # Store None if training fails

print("\nAll ARIMAX models trained (or attempted to train).")

## Forecast future sales with ARIMAX

### Subtask:
Use the trained ARIMAX models to forecast sales revenue for the next 1 year for each dataframe, using the corresponding exogenous data.

**Reasoning**:
Iterate through the trained ARIMAX models and the test time series and exogenous data for each dataframe to generate sales revenue forecasts.

In [ ]:
arimax_forecasts = {}

for name, model in arimax_models.items():
    if model: # Check if model training was successful
        print(f"Generating ARIMAX forecasts for {name}...")
        test_ts = arimax_data[name]['test_ts']
        test_exog = arimax_data[name]['test_exog']

        # Forecast for the length of the test set using the test exogenous data
        forecast = model.forecast(steps=len(test_ts), exog=test_exog)
        arimax_forecasts[name] = forecast
        print(f"Finished forecasting for {name}")
    else:
        print(f"Skipping forecasting for {name} due to failed model training.")
        arimax_forecasts[name] = None

print("\nAll ARIMAX forecasts generated (or skipped).")

## Calculate confidence intervals for ARIMAX forecasts

### Subtask:
Calculate confidence intervals for the ARIMAX forecasted sales revenue.

**Reasoning**:
Iterate through the trained ARIMAX models and their corresponding forecasts to calculate the confidence intervals.

In [ ]:
arimax_confidence_intervals = {}
confidence_level = 0.95 # 95% confidence interval

for name, model in arimax_models.items():
    if model: # Check if model training was successful
        print(f"Calculating confidence intervals for {name}...")
        test_ts = arimax_data[name]['test_ts']
        test_exog = arimax_data[name]['test_exog']
        # Get forecast and confidence intervals
        forecast_results = model.get_forecast(steps=len(test_ts), exog=test_exog)
        confidence_int = forecast_results.conf_int(alpha=1 - confidence_level)

        arimax_confidence_intervals[name] = confidence_int
        print(f"Finished calculating confidence intervals for {name}")
    else:
        print(f"Skipping confidence interval calculation for {name} due to failed model training.")
        arimax_confidence_intervals[name] = None

print("\nConfidence intervals for all ARIMAX forecasts calculated (or skipped).")

## Visualize ARIMAX forecasts

### Subtask:
For each dataframe, visualize the historical data, the ARIMAX forecasted sales revenue, and the confidence interval.

**Reasoning**:
Iterate through each dataframe and plot the historical data, ARIMAX forecasts, and confidence intervals.

In [ ]:
import matplotlib.pyplot as plt

for name in arimax_data.keys():
    if arimax_models[name] and arimax_forecasts[name] is not None and arimax_confidence_intervals[name] is not None:
        plt.figure(figsize=(12, 6))

        # Plot historical data
        plt.plot(arimax_data[name]['train_ts'].index, arimax_data[name]['train_ts'], label='Historical Data')
        plt.plot(arimax_data[name]['test_ts'].index, arimax_data[name]['test_ts'], label='Actual Future Data', color='orange')


        # Plot forecast and confidence intervals
        # Ensure the forecast and confidence interval indices match the test data index
        forecast_index = arimax_data[name]['test_ts'].index
        plt.plot(forecast_index, arimax_forecasts[name], label='ARIMAX Forecast', color='green')
        plt.fill_between(forecast_index,
                         arimax_confidence_intervals[name].iloc[:, 0],
                         arimax_confidence_intervals[name].iloc[:, 1], color='green', alpha=.2, label='95% Confidence Interval')

        plt.title(f'ARIMAX Sales Revenue Forecast for {name}')
        plt.xlabel('Date')
        plt.ylabel('Sales Revenue (USD)')
        plt.legend()
        plt.show()
plt.close()

    else:
        print(f"Skipping visualization for {name} due to failed model training or forecasting.")

print("\nAll ARIMAX forecasts visualized (or skipped).")

## Summarize ARIMAX forecasting results

### Subtask:
Calculate and summarize the performance metrics (MAE, RMSE, MAPE) for the ARIMAX forecasts and discuss the key findings and potential next steps, including comparison with other models.

**Reasoning**:
Calculate MAE, RMSE, and MAPE for the ARIMAX forecasts for each dataframe and display the results in a table. Discuss the performance of the ARIMAX models and suggest next steps, including comparing the ARIMAX results to the ARIMA and XGBoost results.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

arimax_results_summary = {}

def mean_absolute_percentage_error(y_true, y_pred):
    """Calculates Mean Absolute Percentage Error (MAPE)."""
    # Avoid division by zero for actual values that are zero
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # Replace 0s in y_true with a small epsilon to avoid division by zero
    y_true = np.where(y_true == 0, np.finfo(float).eps, y_true)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

for name in arimax_forecasts.keys():
    if arimax_forecasts[name] is not None:
        y_true = arimax_data[name]['test_ts']
        y_pred_series = arimax_forecasts[name]

        # Explicitly set the index of the forecast to match the test data index
        y_pred_series.index = y_true.index

        # Align the series based on their index (which should now match)
        y_true_aligned, y_pred_aligned = y_true.align(y_pred_series, join='inner')

        # Ensure there are samples after alignment
        if len(y_true_aligned) > 0:
            mae = mean_absolute_error(y_true_aligned, y_pred_aligned)
            rmse = np.sqrt(mean_squared_error(y_true_aligned, y_pred_aligned))
            mape = mean_absolute_percentage_error(y_true_aligned, y_pred_aligned)

            arimax_results_summary[name] = {
                'ARIMAX MAE': mae,
                'ARIMAX RMSE': rmse,
                'ARIMAX MAPE': mape
            }
        else:
            print(f"No overlapping dates for {name} after alignment. Skipping metrics calculation.")
            arimax_results_summary[name] = {
                'ARIMAX MAE': np.nan,
                'ARIMAX RMSE': np.nan,
                'ARIMAX MAPE': np.nan
            }
    else:
         arimax_results_summary[name] = {
            'ARIMAX MAE': np.nan,
            'ARIMAX RMSE': np.nan,
            'ARIMAX MAPE': np.nan
        }


arimax_summary_df = pd.DataFrame.from_dict(arimax_results_summary, orient='index')

print("Summary of ARIMAX Forecasting Performance (MAE, RMSE, and MAPE):")
display(arimax_summary_df)

# SUMMARY RESULTS AND DATA ANALYSIS FINDING

In [ ]:
# Combine all summary dataframes
combined_summary_df = summary_df.join(arima_summary_df).join(arimax_summary_df)

# Add AIC and BIC from ARIMA and ARIMAX models
aic_bic_results = {}

for name, model in arima_models.items():
    if model:
        aic_bic_results[name] = {
            'ARIMA AIC': model.aic,
            'ARIMA BIC': model.bic
        }
    else:
        aic_bic_results[name] = {
            'ARIMA AIC': np.nan,
            'ARIMA BIC': np.nan
        }

for name, model in arimax_models.items():
    if model:
        # ARIMAX models from statsmodels have AIC and BIC attributes
        aic_bic_results[name]['ARIMAX AIC'] = model.aic
        aic_bic_results[name]['ARIMAX BIC'] = model.bic
    else:
        aic_bic_results[name]['ARIMAX AIC'] = np.nan
        aic_bic_results[name]['ARIMAX BIC'] = np.nan


aic_bic_df = pd.DataFrame.from_dict(aic_bic_results, orient='index')

# Join the AIC and BIC results to the combined summary
final_summary_df = combined_summary_df.join(aic_bic_df)

print("Comprehensive Model Performance Summary:")
display(final_summary_df)

## Model Comparison Summary for Project Submission

This project explored three different modeling approaches to forecast sales revenue: XGBoost, ARIMA, and ARIMAX. Each model was trained and evaluated on historical sales data, and their performance was compared using various metrics.

**1. XGBoost Model**

*   **Type:** Tree-based ensemble model.
*   **Dependent Variable:** Sales Revenue (USD).
*   **Independent Variables:** Marketing Spend (USD) and Ad Campaign Effectiveness.
*   **Approach:** Trained on the historical data with Marketing Spend and Ad Campaign Effectiveness as features to predict Sales Revenue. Forecasts were generated using the trained model and future values of the independent variables.
*   **Performance:** Performance was evaluated using Mean Absolute Error (MAE), Root Mean Squared Error (RMSE), and Mean Absolute Percentage Error (MAPE) on the test set. The `summary_df` table provides a detailed breakdown of these metrics for each Region and Product Category combination.

**2. ARIMA Model**

*   **Type:** Autoregressive Integrated Moving Average (ARIMA) time series model.
*   **Dependent Variable:** Sales Revenue (USD).
*   **Independent Variables:** None (univariate time series model).
*   **Order:** A fixed order of (2, 1, 0) was used for all time series. This order was chosen as a starting point and could potentially be further optimized for individual time series.
*   **Approach:** Trained on the historical time series data of Sales Revenue. Forecasts were generated based on the historical patterns in the data.
*   **Performance:** Performance was evaluated using MAE, RMSE, and MAPE on the test set. The `arima_summary_df` table shows these metrics. Additionally, AIC and BIC were calculated for each trained ARIMA model, providing insights into the model's fit (lower values generally indicate a better fit). These values are included in the `final_summary_df` table.

**3. ARIMAX Model**

*   **Type:** Autoregressive Integrated Moving Average with Exogenous Variables (ARIMAX) time series model.
*   **Dependent Variable:** Sales Revenue (USD).
*   **Independent Variable (Exogenous):** Marketing Spend (USD).
*   **Order:** A fixed ARIMA order of (2, 1, 0) was used, with 'Marketing Spend (USD)' as the exogenous variable.
*   **Approach:** Trained on the historical time series data of Sales Revenue, incorporating 'Marketing Spend (USD)' as an external factor influencing sales. Forecasts were generated using the trained model and the future values of 'Marketing Spend (USD)' for the forecast period.
*   **Performance:** Performance was evaluated using MAE, RMSE, and MAPE on the test set. The `arimax_summary_df` table presents these metrics. AIC and BIC were also calculated for each trained ARIMAX model, and these are included in the `final_summary_df` table.

**Comparison and Conclusion**

The `final_summary_df` table provides a direct comparison of the MAE, RMSE, and MAPE for all three models across each of the 24 Region and Product Category combinations. By examining this table, we can assess which model performed best for each specific time series based on these error metrics (lower values indicate better performance).

Comparing the ARIMA and ARIMAX results helps to understand the impact of including 'Marketing Spend (USD)' as an exogenous variable. If the ARIMAX model shows significantly better performance metrics and lower AIC/BIC values than the ARIMA model for a particular time series, it suggests that Marketing Spend is a valuable predictor of sales revenue for that category/region.

Comparing the time series models (ARIMA and ARIMAX) with the machine learning model (XGBoost) provides insights into whether capturing temporal dependencies alone (ARIMA), temporal dependencies with an external factor (ARIMAX), or a more flexible machine learning approach with potentially non-linear relationships (XGBoost) is more effective for forecasting sales revenue in different scenarios.

Based on the performance metrics in the `final_summary_df` table, you can identify the most accurate model for each specific Region and Product Category. This information can then be used to inform future forecasting efforts and potentially guide business decisions related to marketing spend and other factors.

**Potential Next Steps**

*   **Analyze `final_summary_df`:** Conduct a detailed analysis of the `final_summary_df` to identify which model performed best (lowest MAE, RMSE, MAPE, AIC, BIC) for each specific Region and Product Category.
*   **Investigate Performance Discrepancies:** For cases where one model significantly outperforms the others, delve deeper into the characteristics of that time series and the models to understand why.
*   **Optimal ARIMA/ARIMAX Order:** If time permits and forecasting accuracy is critical, consider implementing a more rigorous method for determining the optimal ARIMA/ARIMAX order for each time series (e.g., grid search over a range of p, d, q values) instead of using a fixed order.
*   **Ensemble Modeling:** Explore creating ensemble models that combine the forecasts from the best-performing models to potentially achieve even greater accuracy.
*   **Incorporate More Exogenous Variables:** If other relevant external factors are available (e.g., economic indicators, competitor data, seasonal dummy variables), consider including them in the ARIMAX or XGBoost models to potentially improve forecasts.